In [779]:
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn import metrics
import matplotlib.pyplot as plt
from sklearn import tree
import os
import torch 
from decimal import Decimal
from torch.utils.data import Dataset, DataLoader,TensorDataset
import numpy as np
import math
import random
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import matplotlib.pyplot as plt
import multiprocessing
import glob
import os
from scipy import linalg
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from torch.autograd import Variable 

In [780]:
df1=pd.read_csv('data_without_ioa.csv')
df2=pd.read_csv('data_with_ioa.csv')

In [781]:
df1.drop(columns=df1.columns[0], axis=1, inplace=True)
df2.drop(columns=df2.columns[0], axis=1, inplace=True)
df2.drop(['ioa'], axis=1, inplace=True)


df=pd.concat([df1,df2])

In [782]:
#majority under sampling
# df
# data=df[df['label']!=0]
# data_s = resample(df[df['label']==0], replace=False,n_samples=10000,random_state=123)
# df=pd.concat([data,data_s])

In [783]:
#All sampling
data_train_sampled=pd.DataFrame([],columns=df.columns.values,dtype=float)
sampling=2000
for i in df['label'].unique():
    if df[df['label']==i].shape[0]>sampling:
        
        data_s = resample(df[df['label']==i], replace=False,n_samples=sampling,random_state=123)
        data_train_sampled=pd.concat([data_train_sampled,data_s])
    
    else:
        data_s = resample(df[df['label']==i], replace=True,n_samples=sampling,random_state=123) 
        data_train_sampled=pd.concat([data_train_sampled,data_s])

In [784]:
# X,y = df.iloc[:, :-1], df.iloc[:, [-1]]
X, y = data_train_sampled.iloc[:, :-1], data_train_sampled.iloc[:, [-1]]



In [785]:
X

,TimeStamp,Relative Time,srcIP,dstIP,srcPort,dstPort,ipLen,len,fmt,uType,asduType,numix,cot,oa,addr
367939,6.175914,169767.892150,3.232236e+09,3.232236e+09,2404.0,55048.0,760.0,34.0,0.0,1.0,11.0,4.0,20.0,0.0,10.0
1534193,16.464258,71112.556800,3.232236e+09,3.232236e+09,2404.0,47134.0,760.0,42.0,0.0,1.0,7.0,4.0,20.0,0.0,0.0
1427387,11.454472,329490.586085,1.677724e+08,1.677725e+08,2404.0,49716.0,58.0,16.0,1.0,2.0,9.0,1.0,3.0,0.0,55.0
568244,8.155717,57703.035179,1.677724e+08,1.677725e+08,2404.0,49716.0,58.0,16.0,1.0,2.0,9.0,1.0,3.0,0.0,55.0
2197029,6.180456,569030.430954,1.677724e+08,1.677725e+08,2404.0,49716.0,58.0,16.0,1.0,2.0,9.0,1.0,3.0,0.0,55.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
400035,21.175666,196579.662383,3.232239e+09,3.232238e+09,2404.0,61254.0,67.0,25.0,1.0,2.0,36.0,1.0,3.0,0.0,65535.0
399869,21.064536,195908.361039,3.232238e+09,3.232239e+09,61254.0,2404.0,46.0,4.0,2.0,2.0,9.0,1.0,3.0,0.0,55.0
359038,21.163942,23702.428604,3.232239e+09,3.232238e+09,2404.0,61254.0,46.0,4.0,0.0,2.0,9.0,1.0,3.0,0.0,55.0
358923,21.082679,23209.794125,3.232238e+09,3.232239e+09,61254.0,2404.0,46.0,4.0,2.0,2.0,9.0,1.0,3.0,0.0,55.0


In [786]:
y['label'].value_counts()


0.0     2000
7.0     2000
8.0     2000
9.0     2000
10.0    2000
11.0    2000
1.0     2000
2.0     2000
3.0     2000
4.0     2000
5.0     2000
6.0     2000
Name: label, dtype: int64

In [787]:
mm = MinMaxScaler()

X_mm = mm.fit_transform(X) 


df_data=torch.Tensor(X_mm)
df_label=y.values[:,0]

# y=torch.Tensor(y.values)


onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = df_label.reshape(len(df_label), 1)
encoded_label = onehot_encoder.fit_transform(integer_encoded)
encoded_label=torch.Tensor(encoded_label)




In [788]:
X_train, X_test, y_train, y_test = train_test_split(df_data, encoded_label, test_size=0.3,random_state=109) # 70% training and 30% test


In [789]:
train_data = TensorDataset(X_train, y_train)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, shuffle=True, batch_size=1000)#train_data.tensors[0].shape[0])
test_loader = DataLoader(test_data, batch_size=test_data.tensors[0].shape[0])

print("Training data batches:")
for X, y in train_loader:
    print(X.shape, y.shape)
    
print("\nTest data batches:")
for X, y in test_loader:
    print(X.shape, y.shape)

Training data batches:
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([1000, 15]) torch.Size([1000, 12])
torch.Size([800, 15]) torch.Size([800, 12])

Test data batches:
torch.Size([7200, 15]) torch.Size([7200, 12])


In [790]:

#Option 1
in_f=train_data.tensors[0].shape[1]
out_f=train_data.tensors[1].shape[1]

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.input = nn.Linear(in_features=in_f, out_features=128)
        self.hidden_1 = nn.Linear(in_features=128, out_features=64)
        self.hidden_12 = nn.Linear(in_features=64, out_features=32)

        self.output = nn.Linear(in_features=32, out_features=out_f)
        
        
        
    def forward(self, x):
        x = F.sigmoid(self.input(x))
        x = F.sigmoid(self.hidden_1(x))
        x = F.sigmoid(self.hidden_12(x))

    
        return self.output(x)
    
    
model = Net()
print(model)

Net(
  (input): Linear(in_features=15, out_features=128, bias=True)
  (hidden_1): Linear(in_features=128, out_features=64, bias=True)
  (hidden_12): Linear(in_features=64, out_features=32, bias=True)
  (output): Linear(in_features=32, out_features=12, bias=True)
)


In [791]:
#Option 2

class MulticlassClassification(nn.Module):
    def __init__(self, num_feature, num_class):
        super(MulticlassClassification, self).__init__()
        
        self.layer_1 = nn.Linear(num_feature, 512)
        self.layer_2 = nn.Linear(512, 128)
        self.layer_3 = nn.Linear(128, 64)
        self.layer_out = nn.Linear(64, num_class) 
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.3)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.batchnorm2 = nn.BatchNorm1d(128)
        self.batchnorm3 = nn.BatchNorm1d(64)
        
    def forward(self, x):
        x = self.layer_1(x)
        x = self.batchnorm1(x)
        x = self.relu(x)
        
        x = self.layer_2(x)
        x = self.batchnorm2(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_3(x)
        x = self.batchnorm3(x)
        x = self.relu(x)
        x = self.dropout(x)
        
        x = self.layer_out(x)
        
        return x

In [792]:
model = MulticlassClassification(num_feature = in_f, num_class=out_f)

In [793]:
num_epochs = 500
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.005)

for epoch in range(num_epochs):
    for X, y in train_loader:
        preds = model(X)
        preds=torch.softmax(preds,axis=1)
        
        loss = loss_function(preds, y)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        y_labels=torch.argmax(y,axis=1)
        pred_labels=torch.argmax(preds,axis=1)
        
        acc=100 * torch.mean((pred_labels == y_labels).float()).item()
            
#     print(metrics.classification_report(y_labels, pred_labels))

    print("Epoch {}/{}, Loss: {:.3f}, Accuracy: {:.3f}".format(epoch+1,num_epochs, loss.item(), acc))
    
    X , y = next(iter(test_loader))
    preds=torch.softmax(model(X),axis=1)
    y_labels=torch.argmax(y,axis=1)
    pred_labels=torch.argmax(preds,axis=1)
    testacc=100 * torch.mean((pred_labels == y_labels).float()).item()
    print('Test Accuracy:', testacc)
    print(metrics.classification_report(y_labels, pred_labels))


Epoch 1/500, Loss: 2.125, Accuracy: 58.125
Test Accuracy: 57.95833468437195
              precision    recall  f1-score   support

           0       0.99      0.47      0.64       625
           1       0.63      0.36      0.46       584
           2       0.41      0.74      0.52       605
           3       0.00      0.00      0.00       599
           4       0.41      0.43      0.42       630
           5       0.61      0.93      0.74       578
           6       0.79      0.99      0.88       563
           7       0.67      0.53      0.59       621
           8       0.79      0.02      0.05       608
           9       0.85      0.82      0.84       603
          10       0.36      0.72      0.48       584
          11       0.61      1.00      0.76       600

    accuracy                           0.58      7200
   macro avg       0.59      0.58      0.53      7200
weighted avg       0.59      0.58      0.53      7200



/Users/hadeerteryak/miniforge3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hadeerteryak/miniforge3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/hadeerteryak/miniforge3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 2/500, Loss: 1.927, Accuracy: 75.000
Test Accuracy: 72.43055701255798
              precision    recall  f1-score   support

           0       1.00      0.47      0.64       625
           1       0.40      0.80      0.53       584
           2       0.66      0.92      0.77       605
           3       1.00      0.01      0.01       599
           4       0.93      0.39      0.55       630
           5       0.83      0.98      0.90       578
           6       0.85      1.00      0.92       563
           7       0.77      0.93      0.85       621
           8       0.91      0.35      0.51       608
           9       0.96      0.90      0.93       603
          10       0.56      1.00      0.72       584
          11       0.87      0.99      0.93       600

    accuracy                           0.72      7200
   macro avg       0.81      0.73      0.69      7200
weighted avg       0.81      0.72      0.69      7200

Epoch 3/500, Loss: 1.821, Accuracy: 84.875
Test Accuracy:

Epoch 11/500, Loss: 1.763, Accuracy: 86.000
Test Accuracy: 86.06944680213928
              precision    recall  f1-score   support

           0       1.00      0.49      0.66       625
           1       0.77      0.84      0.80       584
           2       0.76      0.94      0.84       605
           3       0.99      0.99      0.99       599
           4       0.99      0.49      0.65       630
           5       0.80      1.00      0.89       578
           6       0.90      1.00      0.95       563
           7       0.90      1.00      0.95       621
           8       0.86      0.64      0.73       608
           9       0.95      1.00      0.97       603
          10       0.67      1.00      0.80       584
          11       0.97      1.00      0.98       600

    accuracy                           0.86      7200
   macro avg       0.88      0.86      0.85      7200
weighted avg       0.88      0.86      0.85      7200

Epoch 12/500, Loss: 1.778, Accuracy: 84.625
Test Accurac

Test Accuracy: 90.55555462837219
              precision    recall  f1-score   support

           0       0.99      0.68      0.81       625
           1       0.74      0.82      0.78       584
           2       0.76      0.94      0.84       605
           3       0.96      1.00      0.98       599
           4       1.00      0.49      0.65       630
           5       0.84      1.00      0.91       578
           6       0.91      0.99      0.95       563
           7       0.94      0.99      0.96       621
           8       0.95      1.00      0.97       608
           9       0.96      1.00      0.98       603
          10       0.97      1.00      0.99       584
          11       0.96      1.00      0.98       600

    accuracy                           0.91      7200
   macro avg       0.92      0.91      0.90      7200
weighted avg       0.92      0.91      0.90      7200

Epoch 21/500, Loss: 1.712, Accuracy: 91.125
Test Accuracy: 90.80555438995361
              precision

...

Epoch 461/500, Loss: 1.687, Accuracy: 93.375
Test Accuracy: 94.1944420337677
              precision    recall  f1-score   support

           0       1.00      0.82      0.90       625
           1       0.98      0.84      0.90       584
           2       0.76      0.94      0.84       605
           3       0.99      1.00      0.99       599
           4       0.97      0.73      0.83       630
           5       0.84      1.00      0.91       578
           6       0.92      1.00      0.96       563
           7       0.99      1.00      0.99       621
           8       0.99      1.00      0.99       608
           9       0.97      1.00      0.99       603
          10       0.99      1.00      0.99       584
          11       0.98      1.00      0.99       600

    accuracy                           0.94      7200
   macro avg       0.95      0.94      0.94      7200
weighted avg       0.95      0.94      0.94      7200

Epoch 462/500, Loss: 1.679, Accuracy: 93.875
Test Accura

              precision    recall  f1-score   support

           0       0.99      0.84      0.91       625
           1       0.99      0.84      0.91       584
           2       0.76      0.94      0.84       605
           3       0.98      1.00      0.99       599
           4       0.98      0.73      0.84       630
           5       0.86      1.00      0.93       578
           6       0.89      1.00      0.94       563
           7       1.00      0.99      0.99       621
           8       0.99      1.00      0.99       608
           9       0.98      1.00      0.99       603
          10       0.99      1.00      1.00       584
          11       0.98      1.00      0.99       600

    accuracy                           0.94      7200
   macro avg       0.95      0.94      0.94      7200
weighted avg       0.95      0.94      0.94      7200

Epoch 471/500, Loss: 1.680, Accuracy: 93.875
Test Accuracy: 94.30555701255798
              precision    recall  f1-score   support



Epoch 479/500, Loss: 1.685, Accuracy: 93.375
Test Accuracy: 94.12500262260437
              precision    recall  f1-score   support

           0       0.97      0.83      0.90       625
           1       0.99      0.84      0.91       584
           2       0.76      0.94      0.84       605
           3       0.99      1.00      0.99       599
           4       0.97      0.73      0.83       630
           5       0.88      1.00      0.93       578
           6       0.88      1.00      0.93       563
           7       0.99      1.00      0.99       621
           8       0.99      1.00      0.99       608
           9       0.98      1.00      0.99       603
          10       0.99      1.00      1.00       584
          11       0.98      0.98      0.98       600

    accuracy                           0.94      7200
   macro avg       0.95      0.94      0.94      7200
weighted avg       0.95      0.94      0.94      7200

Epoch 480/500, Loss: 1.687, Accuracy: 93.125
Test Accur

              precision    recall  f1-score   support

           0       1.00      0.82      0.90       625
           1       0.98      0.84      0.90       584
           2       0.76      0.94      0.84       605
           3       0.98      1.00      0.99       599
           4       0.98      0.73      0.84       630
           5       0.86      1.00      0.92       578
           6       0.89      1.00      0.94       563
           7       0.99      1.00      0.99       621
           8       0.98      1.00      0.99       608
           9       0.98      1.00      0.99       603
          10       0.99      1.00      1.00       584
          11       0.98      1.00      0.99       600

    accuracy                           0.94      7200
   macro avg       0.95      0.94      0.94      7200
weighted avg       0.95      0.94      0.94      7200

Epoch 489/500, Loss: 1.668, Accuracy: 95.125
Test Accuracy: 94.27777528762817
              precision    recall  f1-score   support



Epoch 497/500, Loss: 1.667, Accuracy: 95.250
Test Accuracy: 94.16666626930237
              precision    recall  f1-score   support

           0       0.99      0.82      0.90       625
           1       0.98      0.84      0.90       584
           2       0.76      0.94      0.84       605
           3       0.99      1.00      0.99       599
           4       0.97      0.73      0.83       630
           5       0.84      1.00      0.92       578
           6       0.91      1.00      0.95       563
           7       0.99      1.00      0.99       621
           8       0.98      1.00      0.99       608
           9       0.97      1.00      0.99       603
          10       0.99      1.00      1.00       584
          11       0.99      1.00      0.99       600

    accuracy                           0.94      7200
   macro avg       0.95      0.94      0.94      7200
weighted avg       0.95      0.94      0.94      7200

Epoch 498/500, Loss: 1.665, Accuracy: 95.500
Test Accur

In [794]:
#0 7 8 9 10 11 --> 0 1 2 3 4 5